# Сбор данных со страницы VK с помощью API

In [ ]:
myid = 'ID_USER'

version = '5.199'

with open('token.txt') as f:
    token = f.read()

In [ ]:
import datetime
import sqlite3
import requests
import pandas as pd

In [ ]:
method = 'users.get'
parameters = f'user_ids={myid}'

url = f'https://api.vk.com/method/{method}?{parameters}&v={version}&access_token={token}'
url

'https://api.vk.com/method/users.get?user_ids=227679339&v=5.199&access_token=12a25be512a25be512a25be5c611bce781112a212a25be5744f9c8ca3fc4bfe626cfa04'

In [ ]:
response = requests.get(url)
response.json()

{'response': [{'id': 227679339,
   'first_name': 'Karim',
   'last_name': 'Muratov',
   'can_access_closed': True,
   'is_closed': False}]}

In [ ]:
method = 'wall.get'
parameters = f'owner_id={myid}'
count = 100

url = f'https://api.vk.com/method/{method}?{parameters}&v={version}&count={count}&access_token={token}'
url

'https://api.vk.com/method/wall.get?owner_id=227679339&v=5.199&count=100&access_token=12a25be512a25be512a25be5c611bce781112a212a25be5744f9c8ca3fc4bfe626cfa04'

In [ ]:
response = requests.get(url)

data = []
for item in response.json()['response']['items']:
    post_id = item['id']
    post_time = datetime.datetime.fromtimestamp(item['date'])
    likes = item['likes']['count']

    # Определяем время суток
    if 6 <= post_time.hour < 12:
        time_of_day = 'morning'
    elif 12 <= post_time.hour < 18:
        time_of_day = 'afternoon'
    elif 18 <= post_time.hour < 24:
        time_of_day = 'evening'
    else:
        time_of_day = 'night'

    # Определяем день недели
    day_of_week = post_time.strftime('%a')  # получаем сокращённое название дня
    day_of_week = {'Mon': 'Monday', 'Tue': 'Tuesday', 'Wed': 'Wednesday', 'Thu': 'Thursday', 'Fri': 'Friday', 'Sat': 'Saturday', 'Sun': 'Sunday'}[day_of_week]

    data.append({
        'id_post': post_id,
        'date_time': post_time,
        'likes': likes,
        'times_of_day': time_of_day,
        'day_of_the_week': day_of_week
    })

# Создаём DataFrame
df = pd.DataFrame(data)

# Печатаем DataFrame
print(df)

    id_post           date_time  likes times_of_day day_of_the_week
0      3024 2023-12-08 11:12:46     72      morning          Friday
1      3480 2024-09-22 11:09:35     33      morning          Sunday
2      3479 2024-09-21 16:18:47     24    afternoon        Saturday
3      3476 2024-09-18 02:40:28     69        night       Wednesday
4      3474 2024-09-17 02:26:14     30        night         Tuesday
..      ...                 ...    ...          ...             ...
95     2858 2023-07-08 07:20:42     24      morning        Saturday
96     2852 2023-06-26 09:40:19     51      morning          Monday
97     2840 2023-06-25 05:03:27     96        night          Sunday
98     2839 2023-06-22 05:17:34      5        night        Thursday
99     2831 2023-06-13 09:15:42     63      morning         Tuesday

[100 rows x 5 columns]


# Сохранение данных в csv файл

In [ ]:
# Сохраняем в CSV файл
df.to_csv('posts_data.csv', index=False)

# Создание базы данных SQLite

In [ ]:
# Создание подключения к SQLite базе данных
conn = sqlite3.connect('posts.db')

# Запись данных в SQLite
df.to_sql('posts', conn, if_exists='replace', index=False)

# Закрытие соединения
conn.close()

# SQL запрос для анализа влияния

In [ ]:
# Подключение к базе данных
conn = sqlite3.connect('posts.db')
cursor = conn.cursor()

# SQL-запрос
query = """
WITH PostDifferences AS (
    -- Выбираем необходимые поля и вычисляем разницу в днях между текущей и предыдущей датой поста
    SELECT
        id_post,
        date_time,
        likes,
        times_of_day,
        day_of_the_week,
        JULIANDAY(date_time) - LAG(JULIANDAY(date_time)) OVER (ORDER BY date_time) AS gap_days
    FROM
        posts
)

-- Основной запрос для агрегирования данных по времени суток и дню недели
SELECT
    times_of_day,            -- Время суток
    day_of_the_week,        -- День недели
    AVG(likes) AS avg_likes, -- Среднее количество лайков
    AVG(gap_days) AS avg_gap_days -- Средний промежуток между постами в днях
FROM
    PostDifferences
GROUP BY
    times_of_day,           -- Группируем по времени суток
    day_of_the_week         -- Группируем по дню недели
ORDER BY
    avg_likes DESC;         -- Сортируем по среднему количеству лайков в порядке убывания
"""

# Выполнение запроса и получение результата
cursor.execute(query)
rows = cursor.fetchall()

# Закрытие соединения
cursor.close()
conn.close()

# Вывод результата
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(pd.DataFrame(rows, columns=('Идентификатор поста', 'Время и дата поста', 'Количество лайков', 'Промежуток в днях')))

   Идентификатор поста Время и дата поста  Количество лайков  \
0            afternoon             Sunday         141.333333   
1              morning             Sunday          62.571429   
2            afternoon           Saturday          61.000000   
3            afternoon            Tuesday          57.750000   
4            afternoon           Thursday          57.666667   
5            afternoon             Monday          54.000000   
6                night             Friday          54.000000   
7              morning             Monday          52.800000   
8              morning           Thursday          50.666667   
9              morning           Saturday          49.000000   
10               night            Tuesday          47.750000   
11               night          Wednesday          46.750000   
12             morning             Friday          45.800000   
13             morning            Tuesday          45.000000   
14             morning          Wednesda

# **Вывод**: Время суток с максимальным количеством лайков – это утро и вечер. Воскресенье и суббота приносят больше лайков. Есть заметные различия в среднем промежутке между постами, однако его влияние на количество лайков не столь очевидно.Главным фактором для повышения количества лайков можно считать время суток, особенно утренние и вечерние часы. День недели также играет важную роль, особенно в выходные. Промежутки между постами менее значительно влияют на лайки, хотя их оптимизация также может иметь значение.